# Head

In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/Othercomputers/retep's PC/mamba")

Mounted at /content/gdrive


In [ ]:
%pip install triton causal-conv1d

In [ ]:
%pip install --no-build-isolation --verbose .

In [4]:
import torch, transformers, importlib, InitMamba
from transformers import MambaConfig, MambaForCausalLM, AutoTokenizer

# Lab

In [12]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")
inputs = tokenizer(["Today is Monday, Tomorrow is Tuesday.",
           "Today is Sunday, Tomorrow is Monday.",
           "Today is Sunday, Tomorrow is Monday."],
            max_length = 32,
           padding = 'max_length',
           return_tensors='pt').to('cuda')

In [21]:
importlib.reload(InitMamba)
import InitMamba

model1 = InitMamba.MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf").cuda()
# model1.train()

res1 = model1(**inputs, output_ssm_last_states = True)
ssm_last_states = res1.ssm_last_states
print(ssm_last_states[-1])

tensor([[[ 2.1797e-02, -9.7853e-02,  5.0647e-03,  ...,  7.4099e-02,
          -1.4307e-01, -1.8233e-01],
         [ 7.4937e-03, -1.0742e-02,  5.7249e-04,  ..., -8.5048e-03,
           1.3370e-02,  2.0914e-02],
         [-7.4588e-02,  8.4493e-02, -2.8190e-03,  ..., -3.7165e-02,
           8.2320e-02,  1.0191e-01],
         ...,
         [ 9.4912e-03, -1.3890e-02,  2.5660e-03,  ...,  2.9801e-02,
          -6.6740e-02, -8.9122e-02],
         [-1.1792e-01,  5.8620e-02, -9.2658e-03,  ..., -1.1675e-01,
           2.1422e-01,  2.8709e-01],
         [-4.2002e-03,  6.3560e-03, -9.7592e-04,  ..., -6.9139e-03,
           2.5129e-02,  3.5111e-02]],

        [[ 2.4119e-02, -9.5605e-02,  6.1495e-03,  ...,  7.5622e-02,
          -1.4609e-01, -1.8936e-01],
         [ 8.2960e-03, -1.2948e-02,  1.5956e-04,  ..., -3.7550e-03,
           6.0887e-03,  9.3990e-03],
         [-7.3239e-02,  8.2168e-02, -3.4553e-03,  ..., -3.9539e-02,
           8.6269e-02,  1.0851e-01],
         ...,
         [ 9.7279e-03, -1

In [22]:
res = model1(**inputs, output_ssm_last_states = True, inputs_ssm_states = ssm_last_states)
tokenizer.batch_decode(res.logits.argmax(-1))

[' is Wednesday, Tomorrow is Tuesday,\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>',
 ' is Tuesday, Tomorrow is Monday,\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>',
 ' is Tuesday, Tomorrow is Monday,\n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|

In [8]:
res.logits

tensor([[[-50.6909, -66.3721, -47.7313,  ..., -66.4962, -66.2791, -66.5470],
         [-51.3856, -67.4401, -50.2360,  ..., -67.6237, -67.5241, -67.5715],
         [-30.5409, -48.7618, -29.7472,  ..., -48.6926, -48.7306, -48.9155],
         ...,
         [ -5.1941, -24.0816,  -6.4143,  ..., -24.1864, -24.3238, -24.1755],
         [-34.1231, -55.2161, -34.2415,  ..., -55.1909, -55.1099, -55.3476],
         [-40.5456, -60.5549, -46.9419,  ..., -60.6999, -60.5754, -60.7301]],

        [[-50.5575, -66.2595, -47.6861,  ..., -66.3801, -66.1593, -66.4360],
         [-41.8695, -58.5675, -40.6946,  ..., -58.7701, -58.6721, -58.6985],
         [-39.3041, -57.5434, -38.2461,  ..., -57.5422, -57.4609, -57.6744],
         ...,
         [-13.2786, -32.1403, -14.9096,  ..., -32.1917, -32.2581, -32.2139],
         [-33.7567, -54.3394, -34.8079,  ..., -54.2911, -54.1922, -54.4791],
         [-43.9088, -64.0019, -50.8268,  ..., -64.1493, -64.0093, -64.1931]],

        [[-50.5575, -66.2595, -47.6861,  ...

In [23]:
torch.autograd.grad(outputs=res.logits.mean(), inputs=ssm_last_states[0])

(tensor([[[ 0.0000e+00, -4.1876e-05, -4.3776e-03,  ..., -7.8745e-16,
           -7.7959e-09,  5.1060e-17],
          [ 1.0397e-02, -3.0742e-07, -6.2853e-05,  ..., -6.4466e-06,
           -0.0000e+00,  5.4878e-03],
          [-8.0023e-13,  1.0602e-03,  1.0364e-02,  ...,  4.8393e-25,
            3.7654e-13, -0.0000e+00],
          ...,
          [-1.0494e-05,  3.2344e-19,  7.0474e-04,  ..., -8.3368e-03,
           -3.1304e-09,  3.9292e-11],
          [ 1.0459e-02, -1.0855e-02,  1.0058e-33,  ..., -1.4776e-05,
            0.0000e+00, -0.0000e+00],
          [ 2.6720e-03, -2.4888e-05, -1.8886e-03,  ..., -3.2337e-05,
           -0.0000e+00,  1.4410e-03]],
 
         [[ 0.0000e+00, -4.1802e-05,  1.8784e-03,  ..., -8.0543e-16,
           -7.9648e-09,  5.2226e-17],
          [ 5.0486e-03, -1.3516e-07, -2.7047e-05,  ..., -2.8353e-06,
           -0.0000e+00,  3.1578e-03],
          [-8.0727e-13,  9.9192e-04,  8.8240e-03,  ...,  4.8818e-25,
            3.7984e-13, -0.0000e+00],
          ...,
    

In [ ]:
# Model without initial SSM states gives random results
print(tokenizer.batch_decode(res1.logits.argmax(dim=-1)))

# Model with initial SSM states gives "Sunday"
res2 = model1(input_ids, inputs_ssm_states = ssm_last_states)
print(tokenizer.batch_decode(res2.logits.argmax(dim=-1)))

[', the,\n is Monday']
[', the,\n is Monday']


# Check same

In [ ]:
model2 = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf", config=config)

model1.eval()
model2.eval()

print((model2(input_ids).logits - model1(input_ids).logits).abs().sum())